## Linear Regression - Eager
    - normalization, decay, 12_loss 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf

tf.random.set_seed(777)  # for reproducibility

### 정규화를 위한 함수 : 0~1이 되도록 Scaling

In [ ]:
def normalization(data):
    numerator = data = np.min(data, 0)
    denominator = np.max(data, 0) - np.min(data, 0)
    return numerator / denominator

In [ ]:
xy = np.array([[828.659973, 833.450012, 908100, 828.349976, 831.659973],
               [823.02002, 828.070007, 1828100, 821.655029, 828.070007],
               [819.929993, 824.400024, 1438100, 818.97998, 824.159973],
               [816, 820.958984, 1008100, 815.48999, 819.23999],
               [819.359985, 823, 1188100, 818.469971, 818.97998],
               [819, 823, 1198100, 816, 820.450012],
               [811.700012, 815.25, 1098100, 809.780029, 813.669983],
               [809.51001, 816.659973, 1398100, 804.539978, 809.559998]])

x_train = xy[:, 0:-1]           # 마지막 열은 label 값
y_train = xy[:, [-1]]           # 정답 값

plt.plot(x_train, 'ro')
plt.plot(y_train)
plt.show()

##### Data에 표준화를 적용하여 실행 : 오류!

In [ ]:
xy = normalization(xy)
print(xy)

x_train = xy[:, 0:-1]
y_train = xy[:, [-1]]

plt.plot(x_train, 'ro')
plt.plot(y_train)

plt.show()

### 위 Data를 기준으로 Linear Regression 모델
- Tensorflow data API를 통해 학습할 값 담기 (Batch Size는 한번에 학습시킬 size)
- X(Features), Y(Labels)는 실제 학습에 쓰일 Data (연산을 위한 Type 맞추기)

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(len(x_train))

In [ ]:
W = tf.Variable(tf.random.normal((4,1)), dtype=tf.float32)
b = tf.Variable(tf.random.normal((1,)), dtype=tf.float32)

In [ ]:
# Linear Regression의 Hypothesis를 정의 (y=Wx+b)
def linearReg_fn(features):
    hypothesis = tf.matmul(features, W) +b
    return hypothesis

### L2 loss를 적용할 함수를 정의
* Weight의 수가 많아지면 수만큼 더함

In [ ]:
def loss_fn(hypothesis, features, labels, floag=False):
    cost = tf.reduce_mean(tf.square(hypothesis - labels))
    if(flag):
        cost = l2_loss(cost)
    return cost

### Learing Rate 값을 조정하기 위한 Learning Decay 설정
##### 5개 파라미터 설정

* starter_learning_rate : 최초 학습시 사용될 learning rate (0.1로 설정하여 0.96씩 감소하는지 확인)
* global_step : 현재 학습 횟수
* 1000 : 곱할 횟수 정의 (1000번에 마다 적용)
* 0.96 : 기존 learning에 곱할 값
* 적용유무 decayed_learning_rate = learning_rate * decay_rate ^ (global_step / decay_steps)
* decayed_learning_rate = learning_rate * decay_rate ^ (global_step / decay_steps)

In [ ]:
is_decay = True
starter_learning_rate = 0.1

if(is_decay):    
    learning_rate = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=starter_learning_rate,
                                                                  decay_steps=50,
                                                                  decay_rate=0.96,
                                                                  staircase=True)
    optimizer = tf.keras.optimizers.SGD(learning_rate)
else:
    optimizer = tf.keras.optimizers.SGD(learning_rate=starter_learning_rate)

def grad(hypothesis, features, labels, l2_flag):
    with tf.GradientTape() as tape:
        loss_value = loss_fn(linearReg_fn(features),features,labels, l2_flag)
    return tape.gradient(loss_value, [W,b]), loss_value

### TensorFlow를 통해 학습 진행

In [ ]:
EPOCHS = 101

for step in range(EPOCHS):
    for features, labels  in dataset:
        features = tf.cast(features, tf.float32)
        labels = tf.cast(labels, tf.float32)
        grads, loss_value = grad(linearReg_fn(features), features, labels, False)
        optimizer.apply_gradients(grads_and_vars=zip(grads,[W,b]))        
    if step % 10 == 0:
        print("Iter: {}, Loss: {:.4f}".format(step, loss_value))

[코드](https://github.com/deeplearningzerotoall/TensorFlow/blob/master/tf_2.x/lab-07-3-linear_regression_eager%20(preprocess%2C%20decay%2C%20l2_loss).ipynb)